In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from datetime import date
%matplotlib inline

In [2]:
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = "PK3D1G8EU498KGMS7ZA3"
alpaca_secret_key = "pA/bXokU2aWqa1WT9M1IHdvvQI4gG6ZQhU2wdmfC"

# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [3]:
# Subset your tickers, then pull returns data:
# Set the ticker
ticker = ['SPY', 'AGG','DIA', 'XLY','QQQ', 'GLD', 'SLV', 'VWO','USO', 'EFA']

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp('2015-01-01', tz='America/New_York').isoformat()
today = date.today()
end_date = pd.Timestamp(today.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()

# Get 1 year's worth of historical data for SPY and AGG
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df


# Remove the time component of the data
df.index = df.index.date

In [4]:
df = df.drop(
    columns=['open', 'high', 'low', 'volume'],
    level=1
)

In [5]:
daily_returns=df.pct_change()
daily_returns = daily_returns.dropna()
daily_returns.tail()

,AGG,DIA,EFA,GLD,QQQ,SLV,SPY,USO,VWO,XLY
,close,close,close,close,close,close,close,close,close,close
2020-08-07,-0.001129,0.002373,-0.004381,-0.015734,-0.011235,-0.025316,0.000718,-0.011262,-0.020523,0.001641
2020-08-10,-0.000963,0.013110,0.003300,-0.003354,-0.004494,0.031322,0.002989,0.010385,0.000230,0.011682
2020-08-11,-0.002891,-0.003702,0.003759,-0.053534,-0.018758,-0.135926,-0.008136,-0.010942,-0.004144,-0.001690
2020-08-12,-0.001891,0.010246,0.022160,-0.004667,0.025338,0.002992,0.013821,0.024304,0.012483,0.013118
2020-08-13,-0.003705,-0.002428,-0.004427,0.023501,0.002023,0.086761,-0.001660,-0.005073,-0.004110,-0.000348


In [6]:
daily_returns.columns = daily_returns.columns.droplevel(1)

In [7]:
daily_returns.tail()

,AGG,DIA,EFA,GLD,QQQ,SLV,SPY,USO,VWO,XLY
2020-08-07,-0.001129,0.002373,-0.004381,-0.015734,-0.011235,-0.025316,0.000718,-0.011262,-0.020523,0.001641
2020-08-10,-0.000963,0.013110,0.003300,-0.003354,-0.004494,0.031322,0.002989,0.010385,0.000230,0.011682
2020-08-11,-0.002891,-0.003702,0.003759,-0.053534,-0.018758,-0.135926,-0.008136,-0.010942,-0.004144,-0.001690
2020-08-12,-0.001891,0.010246,0.022160,-0.004667,0.025338,0.002992,0.013821,0.024304,0.012483,0.013118
2020-08-13,-0.003705,-0.002428,-0.004427,0.023501,0.002023,0.086761,-0.001660,-0.005073,-0.004110,-0.000348


In [8]:
# Define a function to chunk the data
def window_data(df, window, feature_col_number, target_col_number):
    """
    This function accepts the column number for the features (X) and the target (y).
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns two numpy arrays of X and y.
    """
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [11]:
# Creating the features (X) and target (y) data using the window_data() function.
window_size = 200

feature_column1 = 0 
target_column1 = 0  
X1, y1 = window_data(daily_returns, window_size, feature_column1, target_column1)
print (f"X1 sample values:{X[:5]}")
print (f"y1 sample values:{y[:5]}")

X1 sample values:[[-1.75746069e-02 -9.81169475e-03  1.26113502e-02  1.76929920e-02
  -8.44988345e-03 -7.64031737e-03 -2.71444083e-03 -5.79007275e-03
  -9.35788950e-03  1.27122902e-02  2.43115852e-03  5.19699069e-03
   1.47717761e-02 -5.53156388e-03  2.48841181e-03 -1.31899153e-02
  -1.32182491e-02  9.54665867e-03 -1.25755025e-02  1.21841155e-02
   1.45142914e-02 -3.56445313e-03  1.03395894e-02 -3.24958774e-03
  -4.47666780e-03  1.08509702e-02  6.28596296e-04  9.44718276e-03
   4.33232007e-03  1.71591992e-03 -4.75827941e-05 -6.66190816e-04
   5.99971430e-03 -2.83996781e-04  2.98281331e-03 -1.08572508e-03
  -9.45134918e-04 -3.40570456e-03  6.21766576e-03 -4.05660377e-03
  -4.30993654e-03  1.14160681e-03 -1.42063002e-02  4.33776750e-03
  -1.63163451e-02 -2.29290663e-03  1.27133148e-02 -6.13200715e-03
   1.36513797e-02 -3.21111910e-03  1.18280604e-02 -4.56187037e-03
   4.48730189e-03 -1.80591199e-03 -5.85602742e-03 -1.46544706e-02
  -2.28432564e-03  2.14341387e-03  1.23468793e-02 -8.739076

In [12]:
window_size = 200

feature_column2 = 1 
feature_column3 = 2 
feature_column4 = 3 
feature_column5 = 4 
feature_column6 = 5 
feature_column7 = 6 
feature_column8 = 7 
feature_column9 = 8 
feature_column10 = 9 

target_column2 = 1  
target_column3 = 2
target_column4 = 3
target_column5 = 4
target_column6 = 5
target_column7 = 6
target_column8 = 7
target_column9 = 8
target_column10 = 9

X2, y2 = window_data(daily_returns, window_size, feature_column2, target_column2)
X3, y3 = window_data(daily_returns, window_size, feature_column3, target_column3)
X4, y4 = window_data(daily_returns, window_size, feature_column4, target_column4)
X5, y5 = window_data(daily_returns, window_size, feature_column5, target_column5)
X6, y6 = window_data(daily_returns, window_size, feature_column6, target_column6)
X7, y7 = window_data(daily_returns, window_size, feature_column7, target_column7)
X8, y8 = window_data(daily_returns, window_size, feature_column8, target_column8)
X9, y9 = window_data(daily_returns, window_size, feature_column9, target_column9)
X10, y10 = window_data(daily_returns, window_size, feature_column10, target_column10)
print (f"X3 sample values:{X[:5]}")
print (f"y3 sample values:{y[:5]}")

X3 sample values:[[-1.75746069e-02 -9.81169475e-03  1.26113502e-02  1.76929920e-02
  -8.44988345e-03 -7.64031737e-03 -2.71444083e-03 -5.79007275e-03
  -9.35788950e-03  1.27122902e-02  2.43115852e-03  5.19699069e-03
   1.47717761e-02 -5.53156388e-03  2.48841181e-03 -1.31899153e-02
  -1.32182491e-02  9.54665867e-03 -1.25755025e-02  1.21841155e-02
   1.45142914e-02 -3.56445313e-03  1.03395894e-02 -3.24958774e-03
  -4.47666780e-03  1.08509702e-02  6.28596296e-04  9.44718276e-03
   4.33232007e-03  1.71591992e-03 -4.75827941e-05 -6.66190816e-04
   5.99971430e-03 -2.83996781e-04  2.98281331e-03 -1.08572508e-03
  -9.45134918e-04 -3.40570456e-03  6.21766576e-03 -4.05660377e-03
  -4.30993654e-03  1.14160681e-03 -1.42063002e-02  4.33776750e-03
  -1.63163451e-02 -2.29290663e-03  1.27133148e-02 -6.13200715e-03
   1.36513797e-02 -3.21111910e-03  1.18280604e-02 -4.56187037e-03
   4.48730189e-03 -1.80591199e-03 -5.85602742e-03 -1.46544706e-02
  -2.28432564e-03  2.14341387e-03  1.23468793e-02 -8.739076

In [15]:
# Splitting Data
# Use last 5 days for testing
#split = int(0.7 * len(X))
X_train1 = X1[:-5]
X_train2 = X2[:-5]
X_train3 = X3[:-5]
X_train4 = X4[:-5]
X_train5 = X5[:-5]
X_train6 = X6[:-5]
X_train7 = X7[:-5]
X_train8 = X8[:-5]
X_train9 = X9[:-5]
X_train10 = X10[:-5]

X_test1 = X1[-5:]
X_test2 = X2[-5:]
X_test3 = X3[-5:]
X_test4 = X4[-5:]
X_test5 = X5[-5:]
X_test6 = X6[-5:]
X_test7 = X7[-5:]
X_test8 = X8[-5:]
X_test9 = X9[-5:]
X_test10 = X10[-5:]

y_train1 = y1[:-5]
y_train2 = y2[:-5]
y_train3 = y3[:-5]
y_train4 = y4[:-5]
y_train5 = y5[:-5]
y_train6 = y6[:-5]
y_train7 = y7[:-5]
y_train8 = y8[:-5]
y_train9 = y9[:-5]
y_train10 = y10[:-5]

y_test1 = y1[-5:]
y_test2 = y2[-5:]
y_test3 = y3[-5:]
y_test4 = y4[-5:]
y_test5 = y5[-5:]
y_test6 = y6[-5:]
y_test7 = y7[-5:]
y_test8 = y8[-5:]
y_test9 = y9[-5:]
y_test10 = y10[-5:]

In [16]:
# Use the MinMaxScaler to scale data between 0 and 1.
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X1)
scaler.fit(X2)
scaler.fit(X3)
scaler.fit(X4)
scaler.fit(X5)
scaler.fit(X6)
scaler.fit(X7)
scaler.fit(X8)
scaler.fit(X9)
scaler.fit(X10)

X_train1 = scaler.transform(X_train1)
X_train2 = scaler.transform(X_train2)
X_train3 = scaler.transform(X_train3)
X_train4 = scaler.transform(X_train4)
X_train5 = scaler.transform(X_train5)
X_train6 = scaler.transform(X_train6)
X_train7 = scaler.transform(X_train7)
X_train8 = scaler.transform(X_train8)
X_train9 = scaler.transform(X_train9)
X_train10 = scaler.transform(X_train10)

X_test1 = scaler.transform(X_test1)
X_test2 = scaler.transform(X_test2)
X_test3 = scaler.transform(X_test3)
X_test4 = scaler.transform(X_test4)
X_test5 = scaler.transform(X_test5)
X_test6 = scaler.transform(X_test6)
X_test7 = scaler.transform(X_test7)
X_test8 = scaler.transform(X_test8)
X_test9 = scaler.transform(X_test9)
X_test10 = scaler.transform(X_test10)

scaler.fit(y1)
scaler.fit(y2)
scaler.fit(y3)
scaler.fit(y4)
scaler.fit(y5)
scaler.fit(y6)
scaler.fit(y7)
scaler.fit(y8)
scaler.fit(y9)
scaler.fit(y10)

y_train1 = scaler.transform(y_train1)
y_train2 = scaler.transform(y_train2)
y_train3 = scaler.transform(y_train3)
y_train4 = scaler.transform(y_train4)
y_train5 = scaler.transform(y_train5)
y_train6 = scaler.transform(y_train6)
y_train7 = scaler.transform(y_train7)
y_train8 = scaler.transform(y_train8)
y_train9 = scaler.transform(y_train9)
y_train10 = scaler.transform(y_train10)

y_test1 = scaler.transform(y_test1)
y_test2 = scaler.transform(y_test2)
y_test3 = scaler.transform(y_test3)
y_test4 = scaler.transform(y_test4)
y_test5 = scaler.transform(y_test5)
y_test6 = scaler.transform(y_test6)
y_test7 = scaler.transform(y_test7)
y_test8 = scaler.transform(y_test8)
y_test9 = scaler.transform(y_test9)
y_test10 = scaler.transform(y_test10)

In [18]:
# Reshape the features for the model
X_train1 = X_train1.reshape((X_train1.shape[0], X_train1.shape[1], 1))
X_train2 = X_train2.reshape((X_train2.shape[0], X_train2.shape[1], 1))
X_train3 = X_train3.reshape((X_train3.shape[0], X_train3.shape[1], 1))
X_train4 = X_train4.reshape((X_train4.shape[0], X_train4.shape[1], 1))
X_train5 = X_train5.reshape((X_train5.shape[0], X_train5.shape[1], 1))
X_train6 = X_train6.reshape((X_train6.shape[0], X_train6.shape[1], 1))
X_train7 = X_train7.reshape((X_train7.shape[0], X_train7.shape[1], 1))
X_train8 = X_train8.reshape((X_train8.shape[0], X_train8.shape[1], 1))
X_train9 = X_train9.reshape((X_train9.shape[0], X_train9.shape[1], 1))
X_train10 = X_train10.reshape((X_train10.shape[0], X_train10.shape[1], 1))

X_test1 = X_test1.reshape((X_test1.shape[0], X_test1.shape[1], 1))
X_test2 = X_test2.reshape((X_test2.shape[0], X_test2.shape[1], 1))
X_test3 = X_test3.reshape((X_test3.shape[0], X_test3.shape[1], 1))
X_test4 = X_test4.reshape((X_test4.shape[0], X_test4.shape[1], 1))
X_test5 = X_test5.reshape((X_test5.shape[0], X_test5.shape[1], 1))
X_test6 = X_test6.reshape((X_test6.shape[0], X_test6.shape[1], 1))
X_test7 = X_test7.reshape((X_test7.shape[0], X_test7.shape[1], 1))
X_test8 = X_test8.reshape((X_test8.shape[0], X_test8.shape[1], 1))
X_test9 = X_test9.reshape((X_test9.shape[0], X_test9.shape[1], 1))
X_test10 = X_test10.reshape((X_test10.shape[0], X_test10.shape[1], 1))
print (f"X_train2 sample values:\n{X_train2[:5]} \n")
print (f"X_test2 sample values:\n{X_test2[:5]}")

X_train2 sample values:
[[[-0.53418474]
  [ 1.15572331]
  [ 0.5330224 ]
  [ 0.58536361]
  [ 0.31722209]
  [ 0.35227861]
  [ 0.38801939]
  [ 0.29635523]
  [ 0.34959062]
  [ 0.49715617]
  [ 0.41613307]
  [ 0.43168093]
  [ 0.55288912]
  [ 0.32246662]
  [ 0.40745633]
  [ 0.24014379]
  [ 0.29428912]
  [ 0.53431066]
  [ 0.26665474]
  [-0.26591961]
  [ 1.4275899 ]
  [ 0.41660332]
  [-0.25482771]
  [ 1.20672943]
  [ 0.34971033]
  [-0.28794819]
  [ 1.24786698]
  [ 0.46833266]
  [-0.33514971]
  [ 0.41787368]
  [ 0.39973483]
  [ 1.21146983]
  [-0.28151512]
  [ 0.39316034]
  [ 0.45988854]
  [ 1.20833561]
  [ 0.40738727]
  [-0.37731014]
  [ 0.48802531]
  [ 0.36050972]
  [ 0.34475918]
  [ 0.42683472]
  [ 0.25085344]
  [ 0.48444536]
  [ 0.23092067]
  [ 0.38546982]
  [ 1.36690229]
  [ 0.32689392]
  [-0.22027691]
  [ 1.12888435]
  [-0.22037083]
  [ 1.14012883]
  [-0.25121905]
  [ 1.17930399]
  [ 0.34999949]
  [ 0.23930434]
  [ 0.38295185]
  [ 0.42287264]
  [ 0.55280099]
  [ 0.30091312]
  [ 0.35952772]


## Build and Train LSTM RNN

In [19]:
# Import required Keras modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [25]:
# Define the LSTM RNN model.
model1 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model1.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train1.shape[1], 1))
    )
model1.add(Dropout(dropout_fraction))
# Layer 2
model1.add(LSTM(units=number_units, return_sequences=True))
model1.add(Dropout(dropout_fraction))
# Layer 3
model1.add(LSTM(units=number_units, return_sequences=True))
model1.add(Dropout(dropout_fraction))
# Layer 4
model1.add(LSTM(units=number_units))
model1.add(Dropout(dropout_fraction))
# Output layer
model1.add(Dense(1))

In [26]:
model2 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model2.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train2.shape[1], 1))
    )
model2.add(Dropout(dropout_fraction))
# Layer 2
model2.add(LSTM(units=number_units, return_sequences=True))
model2.add(Dropout(dropout_fraction))
# Layer 3
model2.add(LSTM(units=number_units, return_sequences=True))
model2.add(Dropout(dropout_fraction))
# Layer 4
model2.add(LSTM(units=number_units))
model2.add(Dropout(dropout_fraction))
# Output layer
model2.add(Dense(1))

In [27]:
model3 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model3.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train3.shape[1], 1))
    )
model3.add(Dropout(dropout_fraction))
# Layer 2
model3.add(LSTM(units=number_units, return_sequences=True))
model3.add(Dropout(dropout_fraction))
# Layer 3
model3.add(LSTM(units=number_units, return_sequences=True))
model3.add(Dropout(dropout_fraction))
# Layer 4
model3.add(LSTM(units=number_units))
model3.add(Dropout(dropout_fraction))
# Output layer
model3.add(Dense(1))

In [28]:
model4 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model4.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train4.shape[1], 1))
    )
model4.add(Dropout(dropout_fraction))
# Layer 2
model4.add(LSTM(units=number_units, return_sequences=True))
model4.add(Dropout(dropout_fraction))
# Layer 3
model4.add(LSTM(units=number_units, return_sequences=True))
model4.add(Dropout(dropout_fraction))
# Layer 4
model4.add(LSTM(units=number_units))
model4.add(Dropout(dropout_fraction))
# Output layer
model4.add(Dense(1))

In [29]:
model5 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model5.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train5.shape[1], 1))
    )
model5.add(Dropout(dropout_fraction))
# Layer 2
model5.add(LSTM(units=number_units, return_sequences=True))
model5.add(Dropout(dropout_fraction))
# Layer 3
model5.add(LSTM(units=number_units, return_sequences=True))
model5.add(Dropout(dropout_fraction))
# Layer 4
model5.add(LSTM(units=number_units))
model5.add(Dropout(dropout_fraction))
# Output layer
model5.add(Dense(1))

In [30]:
model6 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model6.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train6.shape[1], 1))
    )
model6.add(Dropout(dropout_fraction))
# Layer 2
model6.add(LSTM(units=number_units, return_sequences=True))
model6.add(Dropout(dropout_fraction))
# Layer 3
model6.add(LSTM(units=number_units, return_sequences=True))
model6.add(Dropout(dropout_fraction))
# Layer 4
model6.add(LSTM(units=number_units))
model6.add(Dropout(dropout_fraction))
# Output layer
model6.add(Dense(1))

In [31]:
model7 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model7.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train7.shape[1], 1))
    )
model7.add(Dropout(dropout_fraction))
# Layer 2
model7.add(LSTM(units=number_units, return_sequences=True))
model7.add(Dropout(dropout_fraction))
# Layer 3
model7.add(LSTM(units=number_units, return_sequences=True))
model7.add(Dropout(dropout_fraction))
# Layer 4
model7.add(LSTM(units=number_units))
model7.add(Dropout(dropout_fraction))
# Output layer
model7.add(Dense(1))

In [32]:
model8 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model8.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train8.shape[1], 1))
    )
model8.add(Dropout(dropout_fraction))
# Layer 2
model8.add(LSTM(units=number_units, return_sequences=True))
model8.add(Dropout(dropout_fraction))
# Layer 3
model8.add(LSTM(units=number_units, return_sequences=True))
model8.add(Dropout(dropout_fraction))
# Layer 4
model8.add(LSTM(units=number_units))
model8.add(Dropout(dropout_fraction))
# Output layer
model8.add(Dense(1))

In [33]:
model9 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model9.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train9.shape[1], 1))
    )
model9.add(Dropout(dropout_fraction))
# Layer 2
model9.add(LSTM(units=number_units, return_sequences=True))
model9.add(Dropout(dropout_fraction))
# Layer 3
model9.add(LSTM(units=number_units, return_sequences=True))
model9.add(Dropout(dropout_fraction))
# Layer 4
model9.add(LSTM(units=number_units))
model9.add(Dropout(dropout_fraction))
# Output layer
model9.add(Dense(1))

In [34]:
model10 = Sequential()

number_units = 200
dropout_fraction = 0.00000000002

# Layer 1
model10.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train10.shape[1], 1))
    )
model10.add(Dropout(dropout_fraction))
# Layer 2
model10.add(LSTM(units=number_units, return_sequences=True))
model10.add(Dropout(dropout_fraction))
# Layer 3
model10.add(LSTM(units=number_units, return_sequences=True))
model10.add(Dropout(dropout_fraction))
# Layer 4
model10.add(LSTM(units=number_units))
model10.add(Dropout(dropout_fraction))
# Output layer
model10.add(Dense(1))

In [35]:
# Compile the model
model1.compile(optimizer="adam", loss="mean_squared_error")
model2.compile(optimizer="adam", loss="mean_squared_error")
model3.compile(optimizer="adam", loss="mean_squared_error")
model4.compile(optimizer="adam", loss="mean_squared_error")
model5.compile(optimizer="adam", loss="mean_squared_error")
model6.compile(optimizer="adam", loss="mean_squared_error")
model7.compile(optimizer="adam", loss="mean_squared_error")
model8.compile(optimizer="adam", loss="mean_squared_error")
model9.compile(optimizer="adam", loss="mean_squared_error")
model10.compile(optimizer="adam", loss="mean_squared_error")

In [36]:
# Train the model
model1.fit(X_train1, y_train1, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 43s 357ms/step - loss: 0.0090
Epoch 2/10
121/121 [==============================] - 63s 522ms/step - loss: 2.2298e-04
Epoch 3/10
121/121 [==============================] - 67s 555ms/step - loss: 2.3229e-04
Epoch 4/10
121/121 [==============================] - 65s 538ms/step - loss: 2.5751e-04
Epoch 5/10
121/121 [==============================] - 66s 549ms/step - loss: 2.6362e-04
Epoch 6/10
121/121 [==============================] - 68s 562ms/step - loss: 2.6761e-04
Epoch 7/10
121/121 [==============================] - 70s 575ms/step - loss: 2.7600e-04
Epoch 8/10
121/121 [==============================] - 73s 607ms/step - loss: 2.8304e-04
Epoch 9/10
121/121 [==============================] - 75s 621ms/step - loss: 2.8657e-04
Epoch 10/10
121/121 [==============================] - 78s 642ms/step - loss: 2.8770e-04


In [37]:
model2.fit(X_train2, y_train2, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 54s 448ms/step - loss: 0.0175
Epoch 2/10
121/121 [==============================] - 67s 553ms/step - loss: 0.0114
Epoch 3/10
121/121 [==============================] - 72s 599ms/step - loss: 0.0114
Epoch 4/10
121/121 [==============================] - 77s 636ms/step - loss: 0.0114
Epoch 5/10
121/121 [==============================] - 78s 645ms/step - loss: 0.0113
Epoch 6/10
121/121 [==============================] - 79s 655ms/step - loss: 0.0113
Epoch 7/10
121/121 [==============================] - 80s 658ms/step - loss: 0.0112
Epoch 8/10
121/121 [==============================] - 78s 644ms/step - loss: 0.0112
Epoch 9/10
121/121 [==============================] - 65s 533ms/step - loss: 0.0112
Epoch 10/10
121/121 [==============================] - 64s 527ms/step - loss: 0.0112


In [38]:
model3.fit(X_train3, y_train3, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 45s 372ms/step - loss: 0.0298
Epoch 2/10
121/121 [==============================] - 52s 429ms/step - loss: 0.0185
Epoch 3/10
121/121 [==============================] - 57s 471ms/step - loss: 0.0186
Epoch 4/10
121/121 [==============================] - 60s 493ms/step - loss: 0.0186
Epoch 5/10
121/121 [==============================] - 60s 494ms/step - loss: 0.0186
Epoch 6/10
121/121 [==============================] - 64s 532ms/step - loss: 0.0186
Epoch 7/10
121/121 [==============================] - 63s 524ms/step - loss: 0.0185
Epoch 8/10
121/121 [==============================] - 61s 505ms/step - loss: 0.0185
Epoch 9/10
121/121 [==============================] - 63s 520ms/step - loss: 0.0185
Epoch 10/10
121/121 [==============================] - 64s 527ms/step - loss: 0.0185


In [39]:
model4.fit(X_train4, y_train4, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 50s 410ms/step - loss: 0.0114
Epoch 2/10
121/121 [==============================] - 58s 477ms/step - loss: 0.0017
Epoch 3/10
121/121 [==============================] - 61s 502ms/step - loss: 0.0017
Epoch 4/10
121/121 [==============================] - 61s 507ms/step - loss: 0.0018
Epoch 5/10
121/121 [==============================] - 62s 513ms/step - loss: 0.0018
Epoch 6/10
121/121 [==============================] - 61s 507ms/step - loss: 0.0018
Epoch 7/10
121/121 [==============================] - 62s 511ms/step - loss: 0.0017
Epoch 8/10
121/121 [==============================] - 61s 508ms/step - loss: 0.0017
Epoch 9/10
121/121 [==============================] - 65s 534ms/step - loss: 0.0017
Epoch 10/10
121/121 [==============================] - 61s 504ms/step - loss: 0.0017


In [40]:
model5.fit(X_train5, y_train5, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 54s 446ms/step - loss: 0.0198
Epoch 2/10
121/121 [==============================] - 58s 482ms/step - loss: 0.0042
Epoch 3/10
121/121 [==============================] - 59s 484ms/step - loss: 0.0042
Epoch 4/10
121/121 [==============================] - 59s 486ms/step - loss: 0.0042
Epoch 5/10
121/121 [==============================] - 59s 488ms/step - loss: 0.0042
Epoch 6/10
121/121 [==============================] - 60s 493ms/step - loss: 0.0042
Epoch 7/10
121/121 [==============================] - 59s 487ms/step - loss: 0.0042
Epoch 8/10
121/121 [==============================] - 59s 487ms/step - loss: 0.0042
Epoch 9/10
121/121 [==============================] - 60s 494ms/step - loss: 0.0042
Epoch 10/10
121/121 [==============================] - 59s 487ms/step - loss: 0.0042


In [41]:
model6.fit(X_train6, y_train6, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 38s 316ms/step - loss: 0.0195
Epoch 2/10
121/121 [==============================] - 56s 463ms/step - loss: 0.0051
Epoch 3/10
121/121 [==============================] - 59s 486ms/step - loss: 0.0051
Epoch 4/10
121/121 [==============================] - 63s 519ms/step - loss: 0.0051
Epoch 5/10
121/121 [==============================] - 61s 502ms/step - loss: 0.0051
Epoch 6/10
121/121 [==============================] - 61s 507ms/step - loss: 0.0050
Epoch 7/10
121/121 [==============================] - 61s 504ms/step - loss: 0.0050
Epoch 8/10
121/121 [==============================] - 61s 503ms/step - loss: 0.0049
Epoch 9/10
121/121 [==============================] - 62s 510ms/step - loss: 0.0049
Epoch 10/10
121/121 [==============================] - 67s 556ms/step - loss: 0.0049


In [42]:
model7.fit(X_train7, y_train7, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 69s 574ms/step - loss: 0.0195
Epoch 2/10
121/121 [==============================] - 61s 502ms/step - loss: 0.0033
Epoch 3/10
121/121 [==============================] - 67s 555ms/step - loss: 0.0033
Epoch 4/10
121/121 [==============================] - 65s 535ms/step - loss: 0.0033
Epoch 5/10
121/121 [==============================] - 70s 580ms/step - loss: 0.0033
Epoch 6/10
121/121 [==============================] - 66s 545ms/step - loss: 0.0033
Epoch 7/10
121/121 [==============================] - 69s 573ms/step - loss: 0.0033
Epoch 8/10
121/121 [==============================] - 68s 559ms/step - loss: 0.0033
Epoch 9/10
121/121 [==============================] - 70s 576ms/step - loss: 0.0033
Epoch 10/10
121/121 [==============================] - 65s 536ms/step - loss: 0.0033


In [43]:
model8.fit(X_train8, y_train8, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 56s 459ms/step - loss: 0.9534
Epoch 2/10
121/121 [==============================] - 66s 543ms/step - loss: 0.9400
Epoch 3/10
121/121 [==============================] - 68s 565ms/step - loss: 0.9359
Epoch 4/10
121/121 [==============================] - 73s 607ms/step - loss: 0.9336
Epoch 5/10
121/121 [==============================] - 76s 630ms/step - loss: 0.9329
Epoch 6/10
121/121 [==============================] - 66s 544ms/step - loss: 0.9321
Epoch 7/10
121/121 [==============================] - 66s 546ms/step - loss: 0.9312
Epoch 8/10
121/121 [==============================] - 65s 538ms/step - loss: 0.9310
Epoch 9/10
121/121 [==============================] - 65s 533ms/step - loss: 0.9306
Epoch 10/10
121/121 [==============================] - 65s 535ms/step - loss: 0.9313


In [44]:
model9.fit(X_train9, y_train9, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 46s 383ms/step - loss: 0.0099
Epoch 2/10
121/121 [==============================] - 63s 521ms/step - loss: 0.0043
Epoch 3/10
121/121 [==============================] - 61s 506ms/step - loss: 0.0043
Epoch 4/10
121/121 [==============================] - 64s 525ms/step - loss: 0.0042
Epoch 5/10
121/121 [==============================] - 64s 529ms/step - loss: 0.0042
Epoch 6/10
121/121 [==============================] - 61s 504ms/step - loss: 0.0041
Epoch 7/10
121/121 [==============================] - 61s 505ms/step - loss: 0.0041
Epoch 8/10
121/121 [==============================] - 65s 539ms/step - loss: 0.0042
Epoch 9/10
121/121 [==============================] - 63s 517ms/step - loss: 0.0043
Epoch 10/10
121/121 [==============================] - 62s 513ms/step - loss: 0.0043


In [45]:
model10.fit(X_train10, y_train10, epochs=10, shuffle=False, batch_size=10, verbose=1)

Epoch 1/10
121/121 [==============================] - 63s 521ms/step - loss: 0.0142
Epoch 2/10
121/121 [==============================] - 67s 554ms/step - loss: 0.0037
Epoch 3/10
121/121 [==============================] - 70s 579ms/step - loss: 0.0037
Epoch 4/10
121/121 [==============================] - 70s 582ms/step - loss: 0.0037
Epoch 5/10
121/121 [==============================] - 70s 577ms/step - loss: 0.0037
Epoch 6/10
121/121 [==============================] - 66s 542ms/step - loss: 0.0037
Epoch 7/10
121/121 [==============================] - 70s 580ms/step - loss: 0.0037
Epoch 8/10
121/121 [==============================] - 65s 538ms/step - loss: 0.0036
Epoch 9/10
121/121 [==============================] - 64s 530ms/step - loss: 0.0036
Epoch 10/10
121/121 [==============================] - 63s 520ms/step - loss: 0.0036


In [46]:
predicted1= model1.predict(X_test1)
predicted2= model2.predict(X_test2)
predicted3= model3.predict(X_test3)
predicted4= model4.predict(X_test4)
predicted5= model5.predict(X_test5)
predicted6= model6.predict(X_test6)
predicted7= model7.predict(X_test7)
predicted8= model8.predict(X_test8)
predicted9= model9.predict(X_test9)
predicted10= model10.predict(X_test10)

In [56]:
predicted10

array([[0.58662635],
       [0.58676094],
       [0.5869585 ],
       [0.58722144],
       [0.5874878 ]], dtype=float32)

In [57]:
# Recover the original prices instead of the scaled version
predicted_returns1 = scaler.inverse_transform(predicted1)
predicted_returns2 = scaler.inverse_transform(predicted2)
predicted_returns3 = scaler.inverse_transform(predicted3)
predicted_returns4 = scaler.inverse_transform(predicted4)
predicted_returns5 = scaler.inverse_transform(predicted5)
predicted_returns6 = scaler.inverse_transform(predicted6)
predicted_returns7 = scaler.inverse_transform(predicted7)
predicted_returns8 = scaler.inverse_transform(predicted8)
predicted_returns9 = scaler.inverse_transform(predicted9)
predicted_returns10 = scaler.inverse_transform(predicted10)

In [61]:
forecast_agg=pd.DataFrame((predicted_returns1 + 1)).prod()
forecast_dia=pd.DataFrame((predicted_returns2 + 1)).prod()
forecast_efa=pd.DataFrame((predicted_returns3 + 1)).prod()
forecast_gld=pd.DataFrame((predicted_returns4 + 1)).prod()
forecast_qqq=pd.DataFrame((predicted_returns5 + 1)).prod()
forecast_slv=pd.DataFrame((predicted_returns6 + 1)).prod()
forecast_spy=pd.DataFrame((predicted_returns7 + 1)).prod()
forecast_uso=pd.DataFrame((predicted_returns8 + 1)).prod()
forecast_vwo=pd.DataFrame((predicted_returns9 + 1)).prod()
forecast_xly=pd.DataFrame((predicted_returns10 + 1)).prod()

In [64]:
forecast_agg = pd.DataFrame(forecast_agg)
forecast_dia=pd.DataFrame(forecast_dia)
forecast_efa=pd.DataFrame(forecast_efa)
forecast_gld=pd.DataFrame(forecast_gld)
forecast_qqq=pd.DataFrame(forecast_qqq)
forecast_slv=pd.DataFrame(forecast_slv)
forecast_spy=pd.DataFrame(forecast_spy)
forecast_uso=pd.DataFrame(forecast_uso)
forecast_vwo=pd.DataFrame(forecast_vwo)
forecast_xly=pd.DataFrame(forecast_xly)

In [66]:
forecast_agg['Return_Forecast_AGG']=forecast_agg[0]
forecast_dia['Return_Forecast_DIA']=forecast_dia[0]
forecast_efa['Return_Forecast_EFA']=forecast_efa[0]
forecast_gld['Return_Forecast_GLD']=forecast_gld[0]
forecast_qqq['Return_Forecast_QQQ']=forecast_qqq[0]
forecast_slv['Return_Forecast_SLV']=forecast_slv[0]
forecast_spy['Return_Forecast_SPY']=forecast_spy[0]
forecast_uso['Return_Forecast_USO']=forecast_uso[0]
forecast_vwo['Return_Forecast_VWO']=forecast_vwo[0]
forecast_xly['Return_Forecast_XLY']=forecast_xly[0]

In [67]:
forecast_agg = forecast_agg.drop(columns=[0])
forecast_dia = forecast_dia.drop(columns=[0])
forecast_efa = forecast_efa.drop(columns=[0])
forecast_gld = forecast_gld.drop(columns=[0])
forecast_qqq = forecast_qqq.drop(columns=[0])
forecast_slv = forecast_slv.drop(columns=[0])
forecast_spy = forecast_spy.drop(columns=[0])
forecast_uso = forecast_uso.drop(columns=[0])
forecast_vwo = forecast_vwo.drop(columns=[0])
forecast_xly = forecast_xly.drop(columns=[0])

In [68]:
portfolio = pd.concat([forecast_agg,forecast_dia,forecast_efa,forecast_gld,forecast_qqq,forecast_slv,forecast_spy,forecast_uso,forecast_vwo,forecast_xly],axis="columns", join="outer")

In [69]:
portfolio.head()

,Return_Forecast_AGG,Return_Forecast_DIA,Return_Forecast_EFA,Return_Forecast_GLD,Return_Forecast_QQQ,Return_Forecast_SLV,Return_Forecast_SPY,Return_Forecast_USO,Return_Forecast_VWO,Return_Forecast_XLY
0,1.020094,1.017355,0.998162,1.023484,1.01581,1.096069,0.996697,1.215405,1.043718,1.0215


In [70]:
lstm=portfolio.T
lstm

,0
Return_Forecast_AGG,1.020094
Return_Forecast_DIA,1.017355
Return_Forecast_EFA,0.998162
Return_Forecast_GLD,1.023484
Return_Forecast_QQQ,1.015810
Return_Forecast_SLV,1.096069
Return_Forecast_SPY,0.996697
Return_Forecast_USO,1.215405
Return_Forecast_VWO,1.043718
Return_Forecast_XLY,1.021500


In [71]:
lstm = lstm.sort_values(by=[0],ascending=False)
lstm

,0
Return_Forecast_USO,1.215405
Return_Forecast_SLV,1.096069
Return_Forecast_VWO,1.043718
Return_Forecast_GLD,1.023484
Return_Forecast_XLY,1.021500
Return_Forecast_AGG,1.020094
Return_Forecast_DIA,1.017355
Return_Forecast_QQQ,1.015810
Return_Forecast_EFA,0.998162
Return_Forecast_SPY,0.996697


In [72]:
lstm.to_csv('lstm.csv')